<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=a5f6e9189f1b7c49b57d27b8a492155d1c6c1f7ca4c8bdca05d7f810740910d3
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-16 12:41:20
-------------------
qualified stocks: 86
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  1.09 L
Current:  1.40 C
-------------------
Today PnL: 70.38 K (0.5%)
Current PnL: -21.32 L (-14.21%)
CY Booked + Current PnL: -9.42 L (-6.28%)
-------------------
Total profit:  1.78 L
Total loss:  -23.10 L
-------------------
Total Booked + Current PnL: 17.56 L (14.26%)
Total Booked PnL: 38.88 L (31.58%)
Curr Year Booked PnL: 11.90 L (8.49%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 88.51 L (63.15%)
Deployed:  1.23 C
Current:  1.40 C
CAGR/XIRR %: 7.71%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CIPLA,1662.74,-20.12,56.0,X-LC,3.77,213460.0,8960.0,14345.0,-0.04,4.38,6.72,11.39,10.0,0.62,1.53,13.72,X40N,ATH,PHARMA
76,TTKPRESTIG,770.00,96.16,48.0,M-SC,1.67,84482.0,-16295.0,16390.0,0.23,-16.17,19.40,0.09,245.0,-0.99,0.61,10.99,OX40N,NTT,DURABLES
83,VOLTAS,1530.00,1.02,57.0,X-MC,1.25,211845.0,20103.0,17647.0,0.01,10.48,8.33,19.69,99.0,1.14,1.52,17.99,XY25,NTT,AC
33,ICICIGI,2260.25,-13.38,66.0,X-MC,2.08,148962.0,12829.0,18293.0,-0.36,9.42,12.28,22.86,91.0,0.70,1.07,23.23,X40,ATH,INSURANCE
56,RELIANCE,1533.00,-13.66,51.0,X-LC,2.86,215935.0,5569.0,23213.0,0.72,2.65,10.75,13.68,37.0,0.24,1.55,19.61,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
66,SONACOMS,1006.00,-33.65,69.0,M-SC,12.26,82565.0,-18594.0,93488.0,4.32,-18.38,113.23,74.03,202.0,-0.20,0.59,16.42,AR,BTT,AUTO
84,WHIRLPOOL,2270.00,-45.46,49.0,M-SC,4.25,92160.0,662.0,78087.0,3.87,0.72,84.73,86.07,223.0,0.01,0.66,33.83,XR,NTT,DURABLES
49,NESTLEIND,1377.00,-3.64,71.0,X-LC,2.58,296595.0,31169.0,25626.0,3.77,11.74,8.64,21.40,11.0,1.22,2.13,19.17,XY25,NTT,FMCG
57,REPCOHOME,880.00,-56.78,66.0,H-SC,2.70,247509.0,-38996.0,296343.0,3.73,-13.61,119.73,89.82,134.0,-0.13,1.78,30.95,XY24,NTT,FINANCE
82,VBL,671.64,-18.69,52.0,X-LC,10.60,293033.0,-22809.0,135469.0,3.13,-7.22,46.23,35.67,5.0,-0.17,2.11,5.63,X40N,ATH,FMCG


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
27,HATHWAY,31.40,852.90,36.0,H-SC,5.20,82200.0,-26520.0,106202.0,-1.86,-24.39,129.20,73.29,161.0,-0.25,0.59,9.95,XR,NTT,ENTERTAINMENT
81,VALIANTORG,838.00,-281.53,30.0,H-SC,4.19,100878.0,-62727.0,184042.0,-1.54,-38.34,182.44,74.15,139.0,-0.34,0.73,29.31,XR,NTT,CHEMICALS
10,BANDHANBNK,400.00,-13.31,46.0,H-SC,2.90,216142.0,-62417.0,319869.0,-1.16,-22.41,147.99,92.42,151.0,-0.20,1.55,24.91,XY24,NTT,BANKS
15,CAMPUS,393.00,-23.07,58.0,M-SC,2.79,160287.0,-7488.0,63329.0,-1.07,-4.46,39.51,33.28,210.0,-0.12,1.15,24.23,XY24,NTT,FOOTWEAR
31,HINDZINC,730.22,35.55,66.0,M-LC,9.48,221067.0,15991.0,96584.0,-0.97,7.80,43.69,54.89,52.0,0.17,1.59,33.77,X5K,ATH,METALS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WHIRLPOOL,2270.0,-45.46,49.0,M-SC,4.25,92160.0,662.0,78087.0,3.87,0.72,84.73,86.07,223.0,0.01,0.66,33.83,XR,NTT,DURABLES
60,SAMMAANCAP,326.0,-186.36,69.0,M-SC,17.62,149634.0,414.0,143768.0,0.78,0.28,96.08,96.62,208.0,0.00,1.08,61.94,XY25,NTT,FINANCE
59,SAIL,228.0,43.54,50.0,M-MC,12.24,226891.0,1929.0,165721.0,1.13,0.86,73.04,74.53,192.0,0.01,1.63,33.39,XY24,BTT,STEEL


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,108.87,61.0,H-SC,11.91,131625.0,-10854.0,30379.0,0.37,-7.62,23.08,13.70,163.0,-0.36,0.95,55.35,OX40N,NTT,BANKS
76,TTKPRESTIG,770.0,96.16,48.0,M-SC,1.67,84482.0,-16295.0,16390.0,0.23,-16.17,19.40,0.09,245.0,-0.99,0.61,10.99,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.79,50.0,H-SC,1.92,221499.0,-48168.0,84502.0,0.18,-17.86,38.15,13.47,138.0,-0.57,1.59,12.63,XY24,NTT,PAINTS
17,CERA,9475.0,-23.87,40.0,H-SC,2.22,138690.0,-37213.0,79234.0,-0.38,-21.16,57.13,23.89,149.0,-0.47,1.00,19.78,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-24.38,45.0,H-MC,7.16,102344.0,-29491.0,71712.0,0.35,-22.37,70.07,32.03,98.0,-0.41,0.74,14.63,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RECLTD,446.00,47.87,52.0,M-LC,6.14,207488.0,4318.0,37804.0,0.39,2.13,18.22,20.74,55.0,0.11,1.49,7.79,XY25,NTT,FINANCE
59,SAIL,228.00,43.54,50.0,M-MC,12.24,226891.0,1929.0,165721.0,1.13,0.86,73.04,74.53,192.0,0.01,1.63,33.39,XY24,BTT,STEEL
31,HINDZINC,730.22,35.55,66.0,M-LC,9.48,221067.0,15991.0,96584.0,-0.97,7.80,43.69,54.89,52.0,0.17,1.59,33.77,X5K,ATH,METALS
85,WIPRO,420.00,-10.62,60.0,M-LC,5.79,156879.0,5934.0,103524.0,1.13,3.93,65.99,72.51,53.0,0.06,1.13,10.19,XR,NTT,IT
78,UNIONBANK,163.00,-13.18,54.0,M-LC,8.72,157588.0,16748.0,27735.0,-0.60,11.89,17.60,31.59,66.0,0.60,1.13,41.53,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HINDZINC,730.22,35.55,66.0,M-LC,9.48,221067.0,15991.0,96584.0,-0.97,7.80,43.69,54.89,52.0,0.17,1.59,33.77,X5K,ATH,METALS
11,BANKINDIA,190.00,-25.47,61.0,H-MC,13.01,193265.0,13457.0,98952.0,-0.54,7.48,51.20,62.52,88.0,0.14,1.39,44.12,XR,NTT,BANKS
85,WIPRO,420.00,-10.62,60.0,M-LC,5.79,156879.0,5934.0,103524.0,1.13,3.93,65.99,72.51,53.0,0.06,1.13,10.19,XR,NTT,IT
84,WHIRLPOOL,2270.00,-45.46,49.0,M-SC,4.25,92160.0,662.0,78087.0,3.87,0.72,84.73,86.07,223.0,0.01,0.66,33.83,XR,NTT,DURABLES
36,INDIAMART,4810.62,-55.20,41.0,H-SC,8.08,125101.0,1765.0,129867.0,1.32,1.43,103.81,106.72,119.0,0.01,0.90,25.14,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-54.49,14.0,X-LC,5.42,144256.0,-132267.0,242336.0,1.68,-47.83,167.99,39.81,54.0,-0.55,1.04,1.68,XY24,NTT,AUTO
28,HAVELLS,2069.16,-8.16,34.0,X-MC,1.87,227807.0,-19924.0,94973.0,-0.27,-8.04,41.69,30.30,92.0,-0.21,1.64,4.18,X40,ATH,ELECTRICAL
51,QUESS,986.00,-47.66,35.0,X-SC,38.52,52188.0,-12818.0,163745.0,0.00,-19.72,313.76,232.18,198.0,-0.08,0.38,1.77,XY24,NTT,MISC
34,ICICIPRULI,790.00,-23.06,39.0,X-MC,2.21,132368.0,-3420.0,46170.0,-0.39,-2.52,34.88,31.48,107.0,-0.07,0.95,9.29,X40,ATH,INSURANCE
12,BATAINDIA,2096.00,-36.58,40.0,X-SC,4.33,94886.0,-33784.0,79078.0,1.74,-26.26,83.34,35.20,219.0,-0.43,0.68,8.91,X40,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,4671.50,-3.04,44.0,H-LC,0.61,155695.0,-30400.0,77879.0,-0.15,-16.34,50.02,25.51,15.0,-0.39,1.12,14.74,AR,ATH,ELECTRICAL
83,VOLTAS,1530.00,1.02,57.0,X-MC,1.25,211845.0,20103.0,17647.0,0.01,10.48,8.33,19.69,99.0,1.14,1.52,17.99,XY25,NTT,AC
28,HAVELLS,2069.16,-8.16,34.0,X-MC,1.87,227807.0,-19924.0,94973.0,-0.27,-8.04,41.69,30.30,92.0,-0.21,1.64,4.18,X40,ATH,ELECTRICAL
33,ICICIGI,2260.25,-13.38,66.0,X-MC,2.08,148962.0,12829.0,18293.0,-0.36,9.42,12.28,22.86,91.0,0.70,1.07,23.23,X40,ATH,INSURANCE
49,NESTLEIND,1377.00,-3.64,71.0,X-LC,2.58,296595.0,31169.0,25626.0,3.77,11.74,8.64,21.40,11.0,1.22,2.13,19.17,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-54.49,14.0,X-LC,5.42,144256.0,-132267.0,242336.0,1.68,-47.83,167.99,39.81,54.0,-0.55,1.04,1.68,XY24,NTT,AUTO
51,QUESS,986.00,-47.66,35.0,X-SC,38.52,52188.0,-12818.0,163745.0,0.00,-19.72,313.76,232.18,198.0,-0.08,0.38,1.77,XY24,NTT,MISC
73,TCS,4389.96,-29.97,45.0,X-LC,12.33,279161.0,-66795.0,133495.0,0.70,-19.31,47.82,19.28,1.0,-0.50,2.01,3.20,X40,ATH,IT
9,BAJAJHFL,181.50,-14.48,44.0,X-MC,5.82,180170.0,-20336.0,117489.0,-0.25,-10.14,65.21,48.45,90.0,-0.17,1.30,3.90,X40N,ATH,FINANCE
41,ITC,452.00,-38.77,49.0,X-LC,2.66,197312.0,-2826.0,23717.0,0.90,-1.41,12.02,10.44,4.0,-0.12,1.42,4.07,X40,NTT,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-47.66,35.0,X-SC,38.52,52188.0,-12818.0,163745.0,0.00,-19.72,313.76,232.18,198.0,-0.08,0.38,1.77,XY24,NTT,MISC
55,RELAXO,1176.00,-39.20,44.0,X-SC,4.00,82842.0,-62318.0,138247.0,-0.31,-42.93,166.88,52.31,136.0,-0.45,0.60,10.89,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-36.58,40.0,X-SC,4.33,94886.0,-33784.0,79078.0,1.74,-26.26,83.34,35.20,219.0,-0.43,0.68,8.91,X40,NTT,FOOTWEAR
16,CAMS,5211.76,-5.16,50.0,X-SC,1.00,107864.0,5860.0,38065.0,0.42,5.74,35.29,43.06,122.0,0.15,0.78,23.84,X40N,ATH,MISC
32,HONAUT,58357.33,-23.25,44.0,X-SC,2.40,108225.0,-19713.0,66851.0,-0.22,-15.41,61.77,36.84,143.0,-0.29,0.78,10.80,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.96,-29.97,45.0,X-LC,12.33,279161.0,-66795.0,133495.0,0.70,-19.31,47.82,19.28,1.0,-0.50,2.01,3.20,X40,ATH,IT
39,INFY,2275.00,-19.60,45.0,X-LC,7.55,312578.0,-310.0,172012.0,-0.47,-0.10,55.03,54.87,3.0,-0.00,2.25,6.54,X40,BTT,IT
41,ITC,452.00,-38.77,49.0,X-LC,2.66,197312.0,-2826.0,23717.0,0.90,-1.41,12.02,10.44,4.0,-0.12,1.42,4.07,X40,NTT,FMCG
82,VBL,671.64,-18.69,52.0,X-LC,10.60,293033.0,-22809.0,135469.0,3.13,-7.22,46.23,35.67,5.0,-0.17,2.11,5.63,X40N,ATH,FMCG
1,ABB,7934.00,-40.45,48.0,H-LC,7.25,248712.0,-12907.0,132116.0,0.15,-4.93,53.12,45.57,7.0,-0.10,1.79,5.83,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,6616.67,52.0,L-SC,6.12,73670.0,-20140.0,96758.0,0.05,-21.47,131.34,81.67,269.0,-0.21,0.53,42.66,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-43.66,52.0,L-SC,28.80,82752.0,-30797.0,70844.0,1.15,-27.12,85.61,35.27,268.0,-0.43,0.60,102.54,XR,NTT,HOTELS
48,MASFIN,398.61,-20.51,43.0,H-SC,9.47,90405.0,-7575.0,29174.0,0.08,-7.73,32.27,22.05,152.0,-0.26,0.65,31.41,XR,ATH,FINANCE
77,UJJIVANSFB,60.00,108.87,61.0,H-SC,11.91,131625.0,-10854.0,30379.0,0.37,-7.62,23.08,13.70,163.0,-0.36,0.95,55.35,OX40N,NTT,BANKS
68,SURYODAY,240.00,57.89,53.0,H-SC,10.26,144248.0,-34823.0,100555.0,-0.46,-19.45,69.71,36.71,135.0,-0.35,1.04,43.12,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-186.36,69.0,M-SC,17.62,149634.0,414.0,143768.0,0.78,0.28,96.08,96.62,208.0,0.00,1.08,61.94,XY25,NTT,FINANCE
68,SURYODAY,240.00,57.89,53.0,H-SC,10.26,144248.0,-34823.0,100555.0,-0.46,-19.45,69.71,36.71,135.0,-0.35,1.04,43.12,XR,NTT,BANKS
4,ANGELONE,3033.00,14.29,68.0,X-SC,9.40,202540.0,11534.0,46159.0,1.01,6.04,22.79,30.21,157.0,0.25,1.46,28.18,X40N,NTT,FINANCE
63,SHALBY,327.00,1183.89,56.0,M-SC,23.00,174636.0,-6507.0,51972.0,2.92,-3.59,29.76,25.10,235.0,-0.13,1.26,39.95,XY24,NTT,HEALTHCARE
31,HINDZINC,730.22,35.55,66.0,M-LC,9.48,221067.0,15991.0,96584.0,-0.97,7.80,43.69,54.89,52.0,0.17,1.59,33.77,X5K,ATH,METALS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.61
1,25,44.10
2,50,75.91


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.41
LC    33.80
MC    21.83
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.63
X40      14.77
XY25     12.26
XR       10.96
X40N     10.23
AR        8.18
OX40N     7.64
X200      1.83
X5K       1.59
SR        1.14
MH        0.81
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.51
X-LC    22.69
X-MC    16.09
M-SC    12.75
M-LC     5.34
H-LC     4.74
X-SC     4.68
H-MC     3.81
M-MC     1.63
L-SC     1.47
L-LC     1.03
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.21,-3.83,38.05
IT,12.97,-16.64,79.72
FINANCE,12.26,-11.04,61.37
MISC,6.89,-20.05,86.58
BANKS,6.36,-12.77,72.10
PAINTS,5.56,-19.42,37.23
ELECTRICAL,5.33,-11.20,50.22
INSURANCE,3.93,-1.23,39.07
AUTO,2.85,-45.19,114.60


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3315609.0,22
XR,1386506.0,14
AR,1191155.0,9
X40,785120.0,10
X40N,607026.0,8
OX40N,585242.0,10
XY25,477101.0,8
SR,264231.0,2
X5K,96584.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3454316.0,25
M-SC,1270042.0,15
X-MC,1088021.0,11
X-LC,1085077.0,13
X-SC,532145.0,6
H-MC,350121.0,3
H-LC,281363.0,3
M-LC,265647.0,4
L-SC,256940.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1195755.0      6
           XR         828558.0      7
           AR         820664.0      5
M-SC       XY24       688153.0      6
X-MC       XY24       587900.0      4
X-LC       X40        490722.0      5
           XY24       307143.0      2
X-SC       X40N       289322.0      4
H-SC       OX40N      273904.0      4
           SR         264231.0      2
X-MC       X40        215320.0      4
H-LC       AR         209995.0      2
X-LC       X40N       200215.0      3
H-MC       XY24       179457.0      1
L-SC       XR         167602.0      2
X-MC       XY25       167312.0      2
M-MC       XY24       165721.0      1
X-SC       XY24       163745.0      1
M-SC       AR         160496.0      2
           OX40N      150288.0      4
           XY25       143768.0      1
           XR         127337.0      2
X-MC       X40N       117489.0      1
M-LC       XR         103524.0      1
H-MC       XR          98952.0      1
M-LC       X5K         96584.0      1
L-SC       OX40N       89338.0      1
X-LC       XY25        86997.0      3
X-SC       X40         79078.0      1
H-MC       OX40N       71712.0      1
H-LC       X200        71368.0      1
H-SC       MH          71204.0      1
L-MC       XR          60533.0      1
L-LC       XY25        41220.0      1
M-LC       XY25        37804.0      1
           XY24        27735.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
